# IFF+VGD+opt.N model
Inflatable Fat Function, Variable Gravity Distribution and optimized Nipple angle model(膨張可能脂肪体関数+変動重力分布+乳首法線最適化モデル)は，既存のおっぱい関数を発展させたおっぱい関数である．

インタラクティブなグラフはこちら．パラメータを選ぶのにも使える；
[Desmos](https://www.desmos.com/3d/nw4wc8exj4)

仮定や導出に関しては記事を参照すること；
[Qiita](https://qiita.com/anthroteq/items/7dc3dd430a041194fe6f)

## Analytic approach
まずはモデルを解析的に導いていく

注: 明示的に書いていない部分の数式の表示は推奨しない．複雑な式になっているため，texのレンダリングのせいでJupyterが落ちる可能性がある

In [1]:
import sympy as sy

class vector3D(sy.Matrix):
    def __new__(cls, *args, **kwargs):
        instance = sy.Matrix.__new__(cls, *args, **kwargs)
        return instance
    @property
    def x(self):
        return self[0]
    @property
    def y(self):
        return self[1]
    @property
    def z(self):
        return self[2]
    @x.setter
    def x(self, v):
        self[0] = v
    @y.setter
    def y(self, v):
        self[1] = v
    @z.setter
    def z(self, v):
        self[2] = v

In [2]:
x = sy.symbols('x',real=True,positive=True)
y,z = sy.symbols('y,z',real=True)

In [3]:
# 脂肪体関数の定義
a,b = sy.symbols('a,b',real=True)
y_s = sy.symbols('y_s',real=True,positive=True)
f_pre = 1/sy.sqrt(1+b)*sy.sqrt(1+b*x-( (y_s*y)**2+z**2+a*y_s*y*z) )
f_pre

sqrt(-a*y*y_s*z + b*x - y**2*y_s**2 - z**2 + 1)/sqrt(b + 1)

In [4]:
# たわみ前脂肪体関数における乳首座標の計算
x_ray = sy.symbols('x_ray',real=True,positive=True)
y_ray,z_ray = sy.symbols('y_ray,z_ray',real=True)

ans = sy.solve([
    f_pre-x,
    y_ray/x_ray*x-y,
    z_ray/x_ray*x-z
],
    [x,y,z],dict=True)
ans

[{x: x_ray*(b*x_ray - sqrt(4*a*y_ray*y_s*z_ray + b**2*x_ray**2 + 4*b*x_ray**2 + 4*x_ray**2 + 4*y_ray**2*y_s**2 + 4*z_ray**2))/(2*(a*y_ray*y_s*z_ray + b*x_ray**2 + x_ray**2 + y_ray**2*y_s**2 + z_ray**2)),
  y: y_ray*(b*x_ray - sqrt(4*a*y_ray*y_s*z_ray + b**2*x_ray**2 + 4*b*x_ray**2 + 4*x_ray**2 + 4*y_ray**2*y_s**2 + 4*z_ray**2))/(2*(a*y_ray*y_s*z_ray + b*x_ray**2 + x_ray**2 + y_ray**2*y_s**2 + z_ray**2)),
  z: z_ray*(b*x_ray - sqrt(4*a*y_ray*y_s*z_ray + b**2*x_ray**2 + 4*b*x_ray**2 + 4*x_ray**2 + 4*y_ray**2*y_s**2 + 4*z_ray**2))/(2*(a*y_ray*y_s*z_ray + b*x_ray**2 + x_ray**2 + y_ray**2*y_s**2 + z_ray**2))},
 {x: x_ray*(b*x_ray + sqrt(4*a*y_ray*y_s*z_ray + b**2*x_ray**2 + 4*b*x_ray**2 + 4*x_ray**2 + 4*y_ray**2*y_s**2 + 4*z_ray**2))/(2*(a*y_ray*y_s*z_ray + b*x_ray**2 + x_ray**2 + y_ray**2*y_s**2 + z_ray**2)),
  y: y_ray*(b*x_ray + sqrt(4*a*y_ray*y_s*z_ray + b**2*x_ray**2 + 4*b*x_ray**2 + 4*x_ray**2 + 4*y_ray**2*y_s**2 + 4*z_ray**2))/(2*(a*y_ray*y_s*z_ray + b*x_ray**2 + x_ray**2 + y_ray**2*

In [5]:
ans[0][y]<ans[1][y]#正である方が解

y_ray*(b*x_ray - sqrt(4*a*y_ray*y_s*z_ray + b**2*x_ray**2 + 4*b*x_ray**2 + 4*x_ray**2 + 4*y_ray**2*y_s**2 + 4*z_ray**2))/(2*(a*y_ray*y_s*z_ray + b*x_ray**2 + x_ray**2 + y_ray**2*y_s**2 + z_ray**2)) < y_ray*(b*x_ray + sqrt(4*a*y_ray*y_s*z_ray + b**2*x_ray**2 + 4*b*x_ray**2 + 4*x_ray**2 + 4*y_ray**2*y_s**2 + 4*z_ray**2))/(2*(a*y_ray*y_s*z_ray + b*x_ray**2 + x_ray**2 + y_ray**2*y_s**2 + z_ray**2))

In [6]:
x_pre = ans[1][x]
y_pre = ans[1][y]
z_pre = ans[1][z]
P_pre = vector3D([x_pre, y_pre, z_pre])

In [7]:
# 重力分布の定義
L = sy.symbols('L',real=True,positive=True)

## 面積に比例する重力分布項の定義
m = sy.symbols('m',real=True,positive=True)
g = sy.symbols('g',real=True,negative=True)
delta = sy.symbols('delta',real=True,positive=True)
ans = sy.solve(f_pre.subs(y,0)-x,z)
S = ans[0]**2
q_gravity = m*g/L*(1+delta*(S.subs(x,x/L)-1))

## 反重力項の定義
gamma = sy.symbols('gamma',real=True,positive=True)
q_antigravity = gamma/(L**2)*sy.exp(x/L-1)

q = q_gravity + q_antigravity
q

g*m*(delta*(b*x/L - x**2*(b + 1)/L**2) + 1)/L + gamma*exp(-1 + x/L)/L**2

In [8]:
# たわみ関数の計算
E,I = sy.symbols('E,I',real=True,positive=True)
d = sy.symbols('d',real=True,negative=True)
v = sy.Function('v',real=True)

eq = sy.Eq(sy.Derivative(sy.Derivative(E*I*sy.Derivative(sy.Derivative(v(x),x),x),x),x),q)
v = sy.dsolve(eq,ics={v(x).subs(x,0):0,
                  sy.diff(v(x),x,1).subs(x,0):d,
                  sy.diff(v(x),x,2).subs(x,L):0,
                  sy.diff(v(x),x,3).subs(x,L):0}).rhs
v

L**2*gamma*exp(-1 + x/L)/(E*I) - L**2*gamma*exp(-1)/(E*I) + x**2*(L*b*delta*g*m - 3*L*delta*g*m + 6*L*g*m)/(24*E*I) + x*(E*E*I*d - L*gamma)*exp(-1)/(E*I) + g*m*x**4/(24*E*I*L) + x**3*(-L*b*delta*g*m + 2*L*delta*g*m - 6*L*g*m - 6*gamma)/(36*E*I*L) + b*delta*g*m*x**5/(120*E*I*L**2) + delta*g*m*x**6*(-b - 1)/(360*E*I*L**3)

In [9]:
# たわみ後脂肪体関数とたわみ後乳首座標の計算
f_post = L*f_pre.subs({x:x/L,z:z-v})
P_post = vector3D([L*P_pre.x,P_pre.y,P_pre.z+v.subs(x,L*P_pre.x)])

In [10]:
# たわみ後脂肪体関数における乳首座標での法線計算
F = x - f_post
nabla_F = vector3D([sy.diff(F,x,1),sy.diff(F,y,1),sy.diff(F,z,1)])

R = nabla_F.subs({x:P_post.x,y:P_post.y,z:P_post.z})

In [11]:
# 回転軸と回転量の計算
def rodrigues(C,q):
    X = sy.Matrix([
        [C.x**2*(1-sy.cos(q))+sy.cos(q),C.x*C.y*(1-sy.cos(q))-C.z*sy.sin(q),C.x*C.z*(1-sy.cos(q))+C.y*sy.sin(q)],
        [C.x*C.y*(1-sy.cos(q))+C.z*sy.sin(q),C.y**2*(1-sy.cos(q))+sy.cos(q),C.y*C.z*(1-sy.cos(q))-C.x*sy.sin(q)],
        [C.x*C.z*(1-sy.cos(q))-C.y*sy.sin(q),C.y*C.z*(1-sy.cos(q))+C.x*sy.sin(q),C.z**2*(1-sy.cos(q))+sy.cos(q)]
    ])
    return X

C_raw = sy.Matrix.cross(R,vector3D([1,0,0]))
C = C_raw.normalized()

R_normalized = R.normalized()
q = sy.acos(R_normalized.x)

In [12]:
# 乳首関数の定義
t,s,k = sy.symbols('t,s,k',real=True,positive=True)
n_pre = t*sy.exp(-(((y_s*y)**2+z**2+a*y_s*y*z)/s**2)**k)
n_pre

t*exp(-((a*y*y_s*z + y**2*y_s**2 + z**2)/s**2)**k)

In [13]:
# 回転後座標系の計算
co_transformed = rodrigues(C,q) @ vector3D([x,y,z])
x_transformed = co_transformed.x.subs({x:x-P_post.x,y:y-P_post.y,z:z-P_post.z})
y_transformed = co_transformed.y.subs({x:x-P_post.x,y:y-P_post.y,z:z-P_post.z})
z_transformed = co_transformed.z.subs({x:x-P_post.x,y:y-P_post.y,z:z-P_post.z})

In [14]:
# 回転後乳首関数の計算
n_post = t/sy.cos(q)*(n_pre.subs({y:y_transformed,z:z_transformed,t:1}) - (-C.z*sy.sin(q)*(y-P_post.y)+C.y*sy.sin(q)*(z-P_post.z)))

In [15]:
# 乳首領域関数の計算
V = n_pre.subs({y:y_transformed,z:z_transformed,t:1})

In [16]:
# モデルの組み立て
t_1,s_1,k_1 = sy.symbols('t_1,s_1,k_1',real=True)
t_2,s_2,k_2 = sy.symbols('t_2,s_2,k_2',real=True)
eq = sy.Eq(x,f_post+n_post.subs({t:t_1,s:s_1,k:k_1})*V.subs({t:t_1,s:s_1,k:k_1})+n_post.subs({t:t_2,s:s_2,k:k_2})*V.subs({t:t_2,s:s_2,k:k_2}))

In [17]:
# 解析解の保存(必要ならば実行)
with open('pub_01_IFF+VGD+opt.N_analytic-solution.tex','wt') as f:
    f.write(sy.latex(eq))

In [18]:
# もしラムダ関数を更新したいなら実行(cupy)
import dill

eq_zero = eq.rhs-eq.lhs
eq_zero_lambda_cp = sy.lambdify((
    x,y,z,
    x_ray,y_ray,z_ray,
    a,y_s,b,
    m,g,L,E,I,d,delta,gamma,
    t_1,s_1,k_1,
    t_2,s_2,k_2
),eq_zero,modules='cupy')

with open('pub_01_IFF+VGD+opt.N_for_numerical-solution_cp.dill','wb') as f:
    dill.dump(eq_zero_lambda_cp,f)

In [19]:
# もしラムダ関数を更新したいなら実行(numpy)
import dill

eq_zero = eq.rhs-eq.lhs
eq_zero_lambda_np = sy.lambdify((
    x,y,z,
    x_ray,y_ray,z_ray,
    a,y_s,b,
    m,g,L,E,I,d,delta,gamma,
    t_1,s_1,k_1,
    t_2,s_2,k_2
),eq_zero,modules='numpy')

with open('pub_01_IFF+VGD+opt.N_for_numerical-solution_np.dill','wb') as f:
    dill.dump(eq_zero_lambda_np,f)

## Numerical approach
ここからはモデルを数値的に解いて，実際にプロットを行う．とはいえ，解析解は得ているので，パラメータを代入するだけだ

注: GPUでの実行を強くおすすめする．自身の環境で実行したところ，$400 \times 400 \times 400$のグリッドで，CPU，GPUでの実行時間はそれぞれ
+ CPU: 52.5 s ± 757 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
+ GPU: 1.94 s ± 8.03 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

と約2倍の差があった．numpyでの実行についてベクトル化も思いついたが，実施したところ，検証不可能なほど遅くなった．

In [20]:
from mayavi import mlab

In [21]:
# ここにパラメータを設定
params = {
    'x_ray':2.00,#乳首レイ発射座標
    'y_ray':0.00,
    'z_ray':0.50,
    'a':0.60,#脂肪体関数パラメータ
    'y_s':1.1,
    'b':1.00,
    'm':0.006,#たわみ関数パラメータ
    'g':-9.8,
    'L':1.00,
    'E':0.1,
    'I':0.1,
    'd':-0.8,
    'delta':0.28,
    'gamma':0.10,
    't_1':0.07,#乳頭パラメータ
    's_1':0.1,
    'k_1':2.5,
    't_2':0.05,#乳輪パラメータ
    's_2':0.25,
    'k_2':3.3
}

In [22]:
# もしcupyのラムダ関数を使うならこれを実行
import dill
import cupy as cp
import numpy as np

with open('pub_01_IFF+VGD+opt.N_for_numerical-solution_cp.dill','rb') as f:
    breast_func = dill.load(f)

X_c = cp.linspace(0,2,200)
Y_c = cp.linspace(-2,2,400)
Z_c = cp.linspace(-2,2,400)
X_c,Y_c,Z_c = cp.meshgrid(X_c,Y_c,Z_c,indexing='ij')
values_c = cp.asnumpy(breast_func(X_c,Y_c,Z_c,*params.values()))
X = cp.asnumpy(X_c)
Y = cp.asnumpy(Y_c)
Z = cp.asnumpy(Z_c)
values = cp.asnumpy(values_c)

In [23]:
# もしnumpyのラムダ関数を使うならこれを実行
import dill
import numpy as np

with open('pub_01_IFF+VGD+opt.N_for_numerical-solution_np.dill','rb') as f:
    breast_func = dill.load(f)

X = np.linspace(0,2,200)
Y = np.linspace(-2,2,400)
Z = np.linspace(-2,2,400)
X,Y,Z = np.meshgrid(X,Y,Z,indexing='ij')
values = breast_func(X,Y,Z,*params.values())

In [24]:
# プロット
color = [200, 200, 200, 255]
lut = np.zeros((2, 4), dtype=int)
lut[0, :] = color
lut[1, :] = color

mlab.clf()
contour = mlab.contour3d(X,Y,Z, values,contours=[0])
contour.module_manager.scalar_lut_manager.lut.table = lut
contour.module_manager.scalar_lut_manager.lut.number_of_colors = 2
mlab.axes()
mlab.show()

In [25]:
# 3Dで保存したければこちらを実行
from datetime import datetime
import json,os

now = datetime.now().strftime('%Y%m%d-%H%M%S')
filename = 'pub_01_IFF+VGD+opt.N_numerical-solution_'+now

params_txt = {repr(k):v for k,v in params.items()}
with open(filename+'.json','wt') as f:
    json.dump(params_txt,f,indent=4)

mlab.clf()
mlab.contour3d(X, Z, Y, values,contours=[0])
mlab.savefig(filename+'.obj')
mlab.close()
os.remove(filename+'.mtl')#いらない

#Blenderで読み込む場合は，上向きの軸を+Z, 前向きの軸を+Yにすると良い．